Weather Monitoring Display

![Initial Requirement](./Images/ObserverPattern/WeatherMonitoring_1.jpg "Initial Requirement")

![WeatherData Object](./Images/ObserverPattern/WeatherMonitoring_2.jpg "Weather Data Object")

In [4]:
// Initial Implementation of Weather Data

public class WeatherData
{
    // instance variable declaration

    public float GetTemperature () 
    {
        return default(float);
    }

    public float GetHumidity () 
    {
        return default(float);
    }

    public float GetPressure () 
    {
        return default(float);
    }

    public void MeasurementsChanged ()
    {
        float temperature = GetTemperature();
        float humidity = GetHumidity();
        float pressure = GetPressure();

        // currentConditionDisplay.update(temperature, humidity, pressure);
        // statisticsDisplay.update(temperature, humidity, pressure);
        // forecastDisplay.update(temperature, humidity, pressure);
    }
}

*Problems:*
> - By coding to concrete implementations, i.e. currentConditionDisplay, statisticsDisplay and forecastDisplay, we have no way to add or remove other display elements without making changes to the code.
> 
> - The code part where update method is called on different displays seems to be changing. So, we need to encapsulate it.

The **Observer Pattern** defines a one-to-many dependency between objects so that when one object changes state, all of its dependents are notified and updated automatically.

- One *Subject* notifies many *Observers* when something in the subject changes.
- The Observers are dependent on the Subject - when the subject's state changes, the observers are notified.

![Observer Pattern Class Diagram](./Images/ObserverPattern/WeatherMonitoring_3.jpg "Observer Pattern Class Diagram")

**Desing Principle** *Strive for loosely coupled desings between objects that interact.*

- When two objects are loosely coupled, they can interact, but they have very little knowledge of each other.

In Observer pattern context:
- The only thing the subject knows about an observer is that it implements a certain interface (the Observer interface).
- We can add new observers at any time (or even remove it).
- We never need to modify the subject to add new types of observers.
- We can reuse subjects or observers independently of each other.
- Changes to either the subject or an observer will not affect the others.


***Loosely coupled desings allow us to build felxible OO systems that can handle change because they minimize the interdependency between the objects.***

Why Observer pattern for Weather Application?

- Observer pattern has one-to-many relationship between the subject and the observers.

- We have WeatherData object which holds state, i.e., temperature, humidity and pressure. Whenever the state changes, the WeatherData object needs to send the data.

- The data being sent by WeatherData object will be used by the display elements, i.e., current conditions, weather statistics and forecast.

- So we can say, WeatherData object is the Subject and our display elements are the Observers.

- The Observers, i.e., the display elements, implements a common Observer interface and utilize its update method to update the display when they are notfied by the Subject, i.e., WeatherData object about state change.


![Weather Station Desing](./Images/ObserverPattern/WeatherMonitoring_4.jpg "Weather Station Desing")

# Code Implementation

In [1]:
public interface IObserver
{
    public void Update();
}

public interface IDisplayElement
{
    public void Display();
}

public interface ISubject
{
    public void RegisterObserver(IObserver observer);
    public void RemoveObserver(IObserver observer);
    public void NotifyObserver();
}

In [2]:
public class WeatherData : ISubject
{
    private List<IObserver> _observers;
    private float _temperature;
    private float _humidity;
    private float _pressure;

    public WeatherData()
    {
        _observers = new List<IObserver>();
    }

    public float Temperature
    {
        get { return _temperature; }
    }

    public float Humidity
    {
        get { return _humidity; }
    }

    public float Pressure
    {
        get { return _pressure; }
    }

    public void RegisterObserver(IObserver observer)
    {
        _observers.Add(observer);
    }

    public void RemoveObserver(IObserver observer)
    {
        _observers.Remove(observer);
    }

    public void NotifyObserver()
    {
        foreach (IObserver observer in _observers)
        {
            observer.Update();
        }
    }

    public void MeasurementsChanged()
    {
        NotifyObserver();
    }

    // Actually, we will be gettig data from client written API or some web portal
    public void SetMeasurements(float temperature, float humidity, float pressure)
    {
        _temperature = temperature;
        _humidity = humidity;
        _pressure = pressure;

        MeasurementsChanged();
    }
}

In [3]:
public class CurrentConditionsDisplay : IObserver, IDisplayElement
{
    private float _temperature;
    private float _humidity;
    private WeatherData _weatherData;

    public CurrentConditionsDisplay(WeatherData weatherData)
    {
        _weatherData = weatherData;
        _weatherData.RegisterObserver(this);
    }

    public void Update()
    {
        _temperature = _weatherData.Temperature;
        _humidity = _weatherData.Humidity;

        Display();
    }

    public void Display()
    {
        Console.WriteLine($"Current Conditions: {_temperature} F degrees and {_humidity} % humidity");
    }
}

In [4]:
WeatherData weatherData = new WeatherData();

CurrentConditionsDisplay currentConditions = new CurrentConditionsDisplay(weatherData);

weatherData.SetMeasurements(80, 65, 30.4f);
weatherData.SetMeasurements(82, 70, 29.2f);
weatherData.SetMeasurements(78, 90, 29.2f);

Current Conditions: 80 F degrees and 65 % humidity
Current Conditions: 82 F degrees and 70 % humidity
Current Conditions: 78 F degrees and 90 % humidity


In [5]:
public class ForecastDisplay : IObserver, IDisplayElement
{
    private float _currentPressure = 29.92f;
    private float _lastPressure;
    private WeatherData _weatherData;
    
    public ForecastDisplay(WeatherData weatherData)
    {
        _weatherData = weatherData;
        _weatherData.RegisterObserver(this);
    }

    public void Update()
    {
        _lastPressure = _currentPressure;
        _currentPressure = _weatherData.Pressure;

        Display();
    }

    public void Display()
    {
        Console.WriteLine($"Forecast: {_lastPressure} Last Pressure and {_currentPressure} Current Pressure");
    }
}

In [6]:
WeatherData weatherData = new WeatherData();

CurrentConditionsDisplay currentConditions = new CurrentConditionsDisplay(weatherData);
ForecastDisplay forecast = new ForecastDisplay(weatherData);

weatherData.SetMeasurements(80, 65, 30.4f);
weatherData.SetMeasurements(82, 70, 29.2f);
weatherData.SetMeasurements(78, 90, 29.2f);

Current Conditions: 80 F degrees and 65 % humidity
Forecast: 29.92 Last Pressure and 30.4 Current Pressure
Current Conditions: 82 F degrees and 70 % humidity
Forecast: 30.4 Last Pressure and 29.2 Current Pressure
Current Conditions: 78 F degrees and 90 % humidity
Forecast: 29.2 Last Pressure and 29.2 Current Pressure
